_NOTE: This notebook is incomplete in the context that the code is a complete example, but lack descriptions for the steps taken._

In [ ]:
from IPython.display import Image

# Method import to find the least busy quantum computer
from qiskit.providers.ibmq import least_busy

# Method import to monitor monitor Quantum Execution in real time
from qiskit.tools.monitor import job_monitor

from qiskit import (
    Aer,
    execute,
    IBMQ,
    QuantumCircuit)

# As we have saved the account previously, we just load the account.
IBMQ.load_account()
print("Success! Your IBM Q account has now been loaded")

# Variational Quantum Eigensolver or VQE...

... is a quantum hybrid algorithm that finds eigenvalues of a given matrix. A quantum hybrid algorithm functions as a quantum subroutine inside of a classical optimization loop. 
There exist many problems and applications that apply to being solved by VQE. E.g. modeling nature such as chemistry or finding optimal combinations when e.g. finding the shortest route (Traveling Salesman Problem) through cities, countries, mail-routes etc. From a mathematicians point of view, these types of problems and challenges are called combinatorial optimization problems.

Through this notebook, we will go through manually implementing the VQE algorithm as a learning excerise, but it is noteworthy that the [Qiskit Aqua](https://qiskit.org/aqua) module contains an optimized version of the [VQE algorithm](https://qiskit.org/documentation/api/qiskit.aqua.algorithms.adaptive.VQE.html?highlight=vqe#qiskit.aqua.algorithms.adaptive.VQE). 

### Notes

* Hamiltonian, is in _simplified terms_ matrix describing all possible energies of a system. Defining the Hamiltonian enables us to calculate the systems behaviour.

* Eigenvalue is our ground state of the system. The states have energy readings which are individually described by the eigenvectors.


## The variational principle
From [wikipedia](https://en.wikipedia.org/wiki/Variational_method_(quantum_mechanics)), "_In quantum mechanics, the variational method is one way of finding approximations to the lowest energy eigenstate or ground state, and some excited states. This allows calculating approximate wavefunctions such as molecular orbitals._"

This quantum subroutine begins by preparing the quantum state, where a Hamiltonian **H** with eigenstates $\phi\lambda$ and eigenvalues $\lambda$:

$$
H|\Psi\lambda⟩ = \lambda|\Psi\lambda⟩
$$

Then measuring the expectation value.
$$
⟨\Psi\lambda|H|\Psi\lambda⟩
$$

The variational principle enables us to do classical computations running optimization loop for finding the eigenvalue. This is done by rotating the _R_ gates so that expectation value converges towards the loweste energy state. 

## Real world example

In chemistry, finding the minimum eigenvalue of a Hermitian matrix, means that you would have characterized the provided molecules ground state energy.
As such, our algorithm would approximate the energy of a given configuration and by guessing various parameters of a given state, repeatitavly try to minimize the energy with respect to the parameters, we will see convergence towards the lowest possible state for that configuration.

![VQE](../images/vqe/vqe.png)

To elaborate on the illustrated above, we.. 
1. Establish the molecular configuration into qubits.
2. Provide guessed parameteres for how we imagine the states and construct the configuration on the quantum circuit.
3. Observe the energy of the prepared states.
4. Provide updated parameters for next run or if convergence has been reached...
5. ... Bob is your uncle.

# Manually running through VQE of the molecule He-H+

We set our ansatz through the following variables and run it through our simulator or quantum computer. After getting the hang of the procedure, it would be advised to try and implement the classical optimzation part of the algorithm. Continuisly and chronologically, altering  the rotations of the gates to reach convergence of a sensable value.

In [ ]:
q0x0 = 0.0
q1x0 = 0.0

q0z0 = 0.0
q1z0 = 0.0

q0z1 = 0.0
q1z1 = 0.0

q0x1 = 0.0
q1x1 = 0.0

q0z2 = 0.0
q1z2 = 0.0

### The quantum part of the algorithm
1. It sets the rotations-gates with applied rotations.
2. Creates entanglement.
3. Applies the remaining rotations.

In [ ]:
openQASM=f'''
    OPENQASM 2.0;
    include "qelib1.inc";

    qreg q[2];
    creg c[2];

    rx({q0x0}) q[0];
    rx({q1x0}) q[1];
    rz({q0z0}) q[0];
    rz({q1z0}) q[1];

    barrier q[0],q[1];

    h q[0];
    cx q[0],q[1];
    h q[1];
    cx q[1],q[0];

    barrier q[0],q[1];

    rz({q0z1}) q[0];
    rz({q1z1}) q[1];
    rx({q0x1}) q[0];
    rx({q1x1}) q[1];
    rz({q0z2}) q[0];
    rz({q1z2}) q[1];

    measure q[0] -> c[0];
    measure q[1] -> c[1];
'''

# Creates Quantum Circuit from our openQASM-formatted string
circ = QuantumCircuit.from_qasm_str(openQASM)

Drawing out our circuit

In [ ]:
circ.draw(output='mpl')

# Running the circuit on a simulator
We can now setup our simulation environment and feel free to set the amount of shots.

In [ ]:
shots = 1024

AccountProvider = IBMQ.get_provider()
device = AccountProvider.get_backend('ibmq_qasm_simulator')
circ = QuantumCircuit.from_qasm_str(openQASM)
result_1 = execute(circ, device ,shots=shots).result()
counts_1 = result_1.get_counts()
expected_value_1 = (counts_1.get('00', 0) + counts_1.get('01', 0) - counts_1.get('10', 0) - counts_1.get('11', 0)) / shots
print('Expected value for sigma_z I is : %s' % expected_value_1)

The higher the value we can achieve, the closer we are to finding the configuration that represents the ground state of said molecule/atom.

# Running the circuit on a real quantum computer

In [ ]:
AccountProvider = IBMQ.get_provider()

provider = IBMQ.get_provider(hub='ibm-q')

# Method to sort the public provider instances
instances = provider.backends(filters=lambda x: x.configuration().n_qubits >= 5
                                        and not x.configuration().simulator
                                            and x.status().operational==True)          

# Method to choose the least busy device of all the sorted devices
backend = least_busy(instances)
    
# Print which backend that has been chosen
print("The instance \"" + str(backend) + "\" has been loaded as our backend")

In [ ]:
# We excecute the job on the choosen backend
job_2 = execute(circ, backend)

# The job monitor makes it possible for us to monitor our job in real time
job_monitor(job_2)

In [ ]:
job_result_2 = job_2.result()
counts_2 = job_result_2.get_counts(circ)
expected_value_2 = (counts_2.get('00', 0) + counts_2.get('01', 0) - counts_2.get('10', 0) - counts_2.get('11', 0)) / shots
print('Expected value for sigma_z I is : %s' % expected_value_2)